# LLM Training Pipeline

This notebook provides an interactive interface for training a 7B parameter language model through the complete pipeline:

1. **Data Preparation** - Download, clean, and tokenize training data
2. **Pretraining** - Train on large text corpora with curriculum learning
3. **SFT** - Supervised fine-tuning on instruction-response pairs
4. **DPO** - Direct preference optimization for alignment
5. **LoRA** - Optional domain-specific fine-tuning

## Requirements

- **GPU**: NVIDIA A100 80GB recommended, H100 for FP8 support
- **Colab**: Pro/Pro+ recommended for longer training sessions
- **Storage**: Google Drive for persistent checkpoints (recommended)

---
## Step 0: Environment Setup

Run these cells first to set up the training environment. Choose your deployment method below.

In [ ]:
#@title ### 0.1 Detect Environment & Mount Google Drive
#@markdown Run this cell to detect if running in Colab and mount Google Drive for persistent storage.

import os
import sys

# Detect if running in Google Colab
IN_COLAB = 'google.colab' in sys.modules

if IN_COLAB:
    print("Running in Google Colab")
    
    # Mount Google Drive for persistent storage
    from google.colab import drive
    drive.mount('/content/drive')
    
    # Set up persistent directories on Google Drive
    DRIVE_BASE = "/content/drive/MyDrive/llm-training-pipeline"
    os.makedirs(DRIVE_BASE, exist_ok=True)
    os.makedirs(f"{DRIVE_BASE}/checkpoints", exist_ok=True)
    os.makedirs(f"{DRIVE_BASE}/data", exist_ok=True)
    
    print(f"Google Drive mounted. Persistent storage at: {DRIVE_BASE}")
else:
    print("Running locally (not in Colab)")
    DRIVE_BASE = None

In [ ]:
#@title ### 0.2 Clone Repository & Install Dependencies
#@markdown Choose your repository source and install dependencies.

#@markdown ---
#@markdown **Repository Settings:**
REPO_URL = "https://github.com/rmarnold/llm-training-pipeline.git"  #@param {type:"string"}
BRANCH = "main"  #@param {type:"string"}

import os
import subprocess

# Clone or update repository
REPO_DIR = "/content/llm-training-pipeline"

if IN_COLAB:
    if os.path.exists(REPO_DIR):
        print(f"Repository exists. Pulling latest changes...")
        %cd {REPO_DIR}
        !git pull origin {BRANCH}
    else:
        print(f"Cloning repository from {REPO_URL}...")
        !git clone -b {BRANCH} {REPO_URL} {REPO_DIR}
        %cd {REPO_DIR}
    
    # Install core dependencies
    print("\n" + "="*50)
    print("Installing dependencies...")
    print("="*50)
    
    # Install the package
    !pip install -q -e .
    
    # Install flash-attn separately (requires special handling)
    print("\nInstalling flash-attention (this may take a few minutes)...")
    try:
        # Try installing pre-built wheel first (faster)
        !pip install -q flash-attn --no-build-isolation 2>/dev/null || \
         pip install -q flash-attn --no-build-isolation --no-cache-dir
        print("flash-attn installed successfully!")
    except:
        print("Warning: flash-attn installation failed. Training will use standard attention.")
        print("This is OK - training will still work, just slightly slower.")
    
    # Install kernel optimizations (Liger Kernel + Cut Cross-Entropy)
    print("\nInstalling kernel optimizations...")
    !pip install -q liger-kernel cut-cross-entropy 2>/dev/null || true
    print("Kernel optimizations installed (Liger Kernel, Cut Cross-Entropy)")
    
    PROJECT_ROOT = REPO_DIR
    print("\n" + "="*50)
    print("Installation complete!")
    print("="*50)
else:
    # Local development - assume we're in the repo
    PROJECT_ROOT = os.path.dirname(os.getcwd()) if 'notebooks' in os.getcwd() else os.getcwd()
    
print(f"\nProject root: {PROJECT_ROOT}")
os.chdir(PROJECT_ROOT)
sys.path.insert(0, os.path.join(PROJECT_ROOT, 'scripts'))

In [ ]:
#@title ### 0.3 Set Up Persistent Storage (Google Drive)
#@markdown Link checkpoints and data directories to Google Drive for persistence across sessions.

#@markdown ---
USE_DRIVE_STORAGE = True  #@param {type:"boolean"}

import os

if IN_COLAB and USE_DRIVE_STORAGE and DRIVE_BASE:
    print("Setting up persistent storage on Google Drive...")
    
    # Create symlinks for checkpoints and data
    local_dirs = ['checkpoints', 'data', 'logs', 'evals']
    
    for dir_name in local_dirs:
        local_path = os.path.join(PROJECT_ROOT, dir_name)
        drive_path = os.path.join(DRIVE_BASE, dir_name)
        
        # Create drive directory if it doesn't exist
        os.makedirs(drive_path, exist_ok=True)
        
        # Remove local dir if it exists (but not if it's already a symlink)
        if os.path.exists(local_path) and not os.path.islink(local_path):
            # Move existing contents to drive
            if os.listdir(local_path):
                print(f"  Moving existing {dir_name} to Drive...")
                !cp -r {local_path}/* {drive_path}/ 2>/dev/null || true
            !rm -rf {local_path}
        
        # Create symlink
        if not os.path.exists(local_path):
            os.symlink(drive_path, local_path)
            print(f"  {dir_name} -> {drive_path}")
    
    print("\nPersistent storage configured!")
    print("Your checkpoints and data will survive Colab disconnections.")
else:
    print("Using local storage (not persistent in Colab)")
    # Create local directories
    for dir_name in ['checkpoints', 'data', 'logs', 'evals']:
        os.makedirs(os.path.join(PROJECT_ROOT, dir_name), exist_ok=True)

### Alternative: Install from pip (if repo is published)

If the package is published to PyPI or you prefer pip installation:

```python
# From PyPI (when published)
!pip install llm-training-pipeline

# From GitHub directly
!pip install git+https://github.com/rmarnold/llm-training-pipeline.git

# With FP8 support (H100 only)
!pip install "llm-training-pipeline[fp8]"
```

In [ ]:
#@title ### 0.4 Check GPU Availability
#@markdown Verify GPU is available and check for FP8 support.

import torch

print("=" * 50)
print("GPU INFORMATION")
print("=" * 50)

if torch.cuda.is_available():
    gpu_name = torch.cuda.get_device_name(0)
    gpu_memory = torch.cuda.get_device_properties(0).total_memory / (1024**3)
    print(f"GPU: {gpu_name}")
    print(f"Memory: {gpu_memory:.1f} GB")
    
    # Check for H100/FP8 support
    capability = torch.cuda.get_device_capability()
    print(f"Compute Capability: {capability[0]}.{capability[1]}")
    
    if capability[0] >= 9:
        print("FP8 Support: AVAILABLE (H100)")
        RECOMMENDED_PRECISION = "fp8"
    elif capability[0] >= 8:
        print("FP8 Support: Not available (use BF16)")
        RECOMMENDED_PRECISION = "bf16"
    else:
        print("FP8 Support: Not available")
        RECOMMENDED_PRECISION = "fp16"
    
    # Memory recommendation
    if gpu_memory >= 80:
        print(f"\nRecommendation: Full 7B training supported")
    elif gpu_memory >= 40:
        print(f"\nRecommendation: Use gradient checkpointing, smaller batch size")
    else:
        print(f"\nWarning: GPU memory may be insufficient for 7B model")
        print("Consider using LoRA or a smaller model")
else:
    print("WARNING: No GPU detected!")
    print("Training will be extremely slow on CPU.")
    RECOMMENDED_PRECISION = "fp32"

print("=" * 50)

In [ ]:
#@title ### 0.5 Start GPU Keepalive (Prevents Idle Timeout) [DISABLED]
#@markdown **Note:** GPU keepalive is currently disabled. Uncomment the code below to enable if needed.
#@markdown 
#@markdown This starts a background process that periodically pings the GPU to prevent Colab from timing out during long CPU-bound operations (like data cleaning).

# GPU Keepalive is DISABLED - uncomment below to enable
# import subprocess
# import os
# 
# if IN_COLAB:
#     # Kill any existing keepalive process
#     !pkill -f gpu_keepalive.py 2>/dev/null || true
#     
#     # Start GPU keepalive in background
#     keepalive_script = os.path.join(PROJECT_ROOT, 'scripts', 'gpu_keepalive.py')
#     
#     if os.path.exists(keepalive_script):
#         process = subprocess.Popen(
#             ['python', keepalive_script],
#             stdout=subprocess.DEVNULL,
#             stderr=subprocess.DEVNULL,
#             start_new_session=True
#         )
#         print(f"GPU Keepalive started (PID: {process.pid})")
#         print("  - Checks GPU every 60 seconds")
#         print("  - Sends keepalive spike if idle for 5+ minutes")
#         print("  - Prevents Colab idle timeout during CPU-bound tasks")
#         print(f"\nTo stop: !pkill -f gpu_keepalive.py")
#     else:
#         print("Warning: gpu_keepalive.py not found. Run 'git pull' to get latest code.")
# else:
#     print("GPU keepalive not needed outside of Colab")

print("GPU Keepalive: DISABLED")
print("To enable, uncomment the code in this cell.")

In [ ]:
#@title ### 0.5 Training Configuration {run: "auto"}
#@markdown Adjust these settings based on your GPU and requirements.

#@markdown ---
#@markdown **General Settings:**
use_fp8 = "auto"  #@param ["auto", "true", "false"]
seed = 42  #@param {type:"integer"}
enable_oom_recovery = True  #@param {type:"boolean"}

#@markdown ---
#@markdown **Kernel Optimizations (Recommended):**
use_liger_kernel = True  #@param {type:"boolean"}
#@markdown *Liger Kernel: ~20% speedup, ~60% memory reduction*
use_cce = True  #@param {type:"boolean"}
#@markdown *Cut Cross-Entropy: ~95% memory reduction on loss computation*

#@markdown ---
#@markdown **Data Cleaning:**
pipeline_mode = "native"  #@param ["native", "legacy"]
#@markdown *Pipeline modes:*
#@markdown - **native**: Uses datatrove's optimized pipeline (3-5x faster, recommended)
#@markdown - **legacy**: Uses multiprocessing Pool (fallback if native fails)

use_full_clean = False  #@param {type:"boolean"}
#@markdown *Full clean uses plsfix (Rust) for Unicode/mojibake fixing. Slower but more thorough.*

quality_filter_mode = "default"  #@param ["default", "fast", "no-repetition", "no-fineweb"]
#@markdown *Quality filter modes:*
#@markdown - **default**: All filters (GopherQuality + FineWeb + GopherRepetition)
#@markdown - **fast**: Only GopherQuality (~3x faster, basic filtering)
#@markdown - **no-repetition**: Skip n-gram analysis (~2x faster, may miss spam)
#@markdown - **no-fineweb**: Skip line structure checks (~15% faster)

#@markdown ---
#@markdown **Pretraining:**
pretrain_max_steps = 100000  #@param {type:"integer"}
pretrain_save_steps = 1000  #@param {type:"integer"}
pretrain_eval_steps = 1000  #@param {type:"integer"}

#@markdown ---
#@markdown **SFT:**
sft_max_steps = 5000  #@param {type:"integer"}
sft_save_steps = 500  #@param {type:"integer"}

#@markdown ---
#@markdown **DPO:**
dpo_max_steps = 2000  #@param {type:"integer"}
dpo_save_steps = 200  #@param {type:"integer"}

# Build config dict
CONFIG = {
    'use_fp8': None if use_fp8 == "auto" else (use_fp8 == "true"),
    'seed': seed,
    'enable_oom_recovery': enable_oom_recovery,
    'use_liger_kernel': use_liger_kernel,
    'use_cce': use_cce,
    'pipeline_mode': pipeline_mode,
    'use_full_clean': use_full_clean,
    'quality_filter_mode': quality_filter_mode,
    'pretrain_max_steps': pretrain_max_steps,
    'pretrain_save_steps': pretrain_save_steps,
    'pretrain_eval_steps': pretrain_eval_steps,
    'sft_max_steps': sft_max_steps,
    'sft_save_steps': sft_save_steps,
    'dpo_max_steps': dpo_max_steps,
    'dpo_save_steps': dpo_save_steps,
}

print("Configuration:")
for key, value in CONFIG.items():
    print(f"  {key}: {value}")

if use_liger_kernel or use_cce:
    print("\nKernel Optimizations Enabled:")
    if use_liger_kernel:
        print("  - Liger Kernel: Fused Triton kernels for ~20% speedup")
    if use_cce:
        print("  - Cut Cross-Entropy: Memory-efficient loss (~95% reduction)")

print(f"\nPipeline Mode: {'NATIVE DATATROVE (optimized)' if pipeline_mode == 'native' else 'LEGACY (multiprocessing)'}")
print(f"Data Cleaning Mode: {'FULL (plsfix)' if use_full_clean else 'FAST (skip Unicode fixing)'}")
print(f"Quality Filter Mode: {quality_filter_mode}")

---
## Pre-flight Validation

Before starting training, validate that all prerequisites are in place.

In [ ]:
# Run pre-flight checks for a specific stage
# Options: 'pretrain', 'sft', 'dpo', 'lora'

STAGE_TO_CHECK = 'pretrain'  # Change this to check different stages

!python scripts/preflight_check.py {STAGE_TO_CHECK}

In [ ]:
# Run all pre-flight checks
!python scripts/preflight_check.py --all

---
## Stage 1: Data Preparation

Download, clean, and prepare training data. Skip this section if data is already prepared.

In [ ]:
# Step 1.1: Download raw data
# This downloads data from configured sources (HuggingFace, etc.)

!python scripts/01_download_data.py

In [ ]:
# Step 1.2: Clean and deduplicate data
# Removes duplicates, filters low-quality content

clean_cmd = "python scripts/02_clean_deduplicate_optimized.py"

# Pipeline mode (native datatrove vs legacy multiprocessing)
pipeline_mode = CONFIG.get('pipeline_mode', 'native')
if pipeline_mode == 'native':
    clean_cmd += " --native-pipeline"
else:
    clean_cmd += " --legacy"

# Unicode cleaning mode
if CONFIG.get('use_full_clean', False):
    clean_cmd += " --full-clean"

# Quality filter mode
filter_mode = CONFIG.get('quality_filter_mode', 'default')
if filter_mode == 'fast':
    clean_cmd += " --fast-quality"
elif filter_mode == 'no-repetition':
    clean_cmd += " --no-repetition-filter"
elif filter_mode == 'no-fineweb':
    clean_cmd += " --no-fineweb-filter"
# 'default' uses all filters, no extra flags needed

print(f"Cleaning command: {clean_cmd}")
print(f"Pipeline mode: {'NATIVE DATATROVE (optimized)' if pipeline_mode == 'native' else 'LEGACY (multiprocessing)'}")
print(f"Unicode cleaning: {'FULL (plsfix)' if CONFIG.get('use_full_clean') else 'FAST (skip Unicode fixing)'}")
print(f"Quality filters: {filter_mode}")
print()

!{clean_cmd}

In [ ]:
# Step 1.3: Tokenize and pack sequences
# Creates packed sequences for efficient training

!python scripts/03_tokenize_and_pack.py

In [ ]:
# Step 1.4: Initialize model
# Creates the initial 7B model checkpoint

!python scripts/04_init_model.py

In [ ]:
# Verify data preparation
import os

paths_to_check = [
    ('Tokenizer', 'configs/tokenizer'),
    ('Initial model', 'checkpoints/init'),
    ('Training data', 'data/packed/train'),
    ('Validation data', 'data/packed/val'),
]

print("Data preparation status:")
print("=" * 50)
all_ready = True
for name, path in paths_to_check:
    exists = os.path.exists(path)
    status = "OK" if exists else "MISSING"
    print(f"  {name}: {status}")
    all_ready = all_ready and exists

print("=" * 50)
if all_ready:
    print("All data preparation complete! Ready for pretraining.")
else:
    print("Some data is missing. Run the preparation steps above.")

---
## Stage 2: Pretraining

Train the base model on large text corpora. This is the longest stage.

**Estimated time:** 25-50 hours depending on GPU (H100 FP8 fastest)

In [ ]:
# Build pretraining command
pretrain_cmd = "python scripts/05_pretrain.py"

if CONFIG['use_fp8'] is True:
    pretrain_cmd += " --fp8"
elif CONFIG['use_fp8'] is False:
    pretrain_cmd += " --no-fp8"

pretrain_cmd += f" --max_steps {CONFIG['pretrain_max_steps']}"
pretrain_cmd += f" --save_steps {CONFIG['pretrain_save_steps']}"
pretrain_cmd += f" --eval_steps {CONFIG['pretrain_eval_steps']}"
pretrain_cmd += f" --seed {CONFIG['seed']}"

if CONFIG['enable_oom_recovery']:
    pretrain_cmd += " --enable-oom-recovery"

# Kernel optimizations (enabled by default)
if CONFIG.get('use_liger_kernel', True):
    pretrain_cmd += " --use-liger-kernel"
else:
    pretrain_cmd += " --no-liger-kernel"

if CONFIG.get('use_cce', True):
    pretrain_cmd += " --use-cce"
else:
    pretrain_cmd += " --no-cce"

print("Pretraining command:")
print(pretrain_cmd)

if CONFIG.get('use_liger_kernel') or CONFIG.get('use_cce'):
    print("\nKernel optimizations enabled for faster training!")

In [ ]:
# Start pretraining
# This will take a long time - monitor progress in the output

!{pretrain_cmd}

In [ ]:
# Resume pretraining from checkpoint (if interrupted)
# Uncomment and modify the checkpoint path as needed

# CHECKPOINT_PATH = "checkpoints/pretrain/checkpoint-5000"
# !python scripts/05_pretrain.py --resume_from_checkpoint {CHECKPOINT_PATH}

---
## Stage 3: Supervised Fine-Tuning (SFT)

Fine-tune on instruction-response pairs to create a helpful assistant.

**Estimated time:** 2-5 hours

In [ ]:
# Prepare SFT data (if not already done)
!python scripts/06_prepare_sft_data.py

In [ ]:
# Verify pretrained checkpoint exists
import os

if os.path.exists('checkpoints/pretrain_final'):
    print("Pretrained checkpoint found. Ready for SFT.")
else:
    print("ERROR: Pretrained checkpoint not found!")
    print("Complete pretraining before starting SFT.")

In [ ]:
# Build SFT command
sft_cmd = "python scripts/07_sft.py"

if CONFIG['use_fp8'] is True:
    sft_cmd += " --fp8"
elif CONFIG['use_fp8'] is False:
    sft_cmd += " --no-fp8"

sft_cmd += f" --max_steps {CONFIG['sft_max_steps']}"
sft_cmd += f" --save_steps {CONFIG['sft_save_steps']}"
sft_cmd += f" --seed {CONFIG['seed']}"

if CONFIG['enable_oom_recovery']:
    sft_cmd += " --enable-oom-recovery"

print("SFT command:")
print(sft_cmd)

In [ ]:
# Start SFT training
!{sft_cmd}

---
## Stage 4: Direct Preference Optimization (DPO)

Align the model with human preferences using chosen/rejected response pairs.

**Estimated time:** 1-3 hours

In [ ]:
# Prepare DPO data
!python scripts/08_prepare_dpo_data.py

In [ ]:
# Verify SFT checkpoint exists
import os

if os.path.exists('checkpoints/sft_final'):
    print("SFT checkpoint found. Ready for DPO.")
else:
    print("ERROR: SFT checkpoint not found!")
    print("Complete SFT before starting DPO.")

In [ ]:
# Build DPO command
dpo_cmd = "python scripts/09_dpo.py"

if CONFIG['use_fp8'] is True:
    dpo_cmd += " --fp8"
elif CONFIG['use_fp8'] is False:
    dpo_cmd += " --no-fp8"

dpo_cmd += f" --max_steps {CONFIG['dpo_max_steps']}"
dpo_cmd += f" --save_steps {CONFIG['dpo_save_steps']}"
dpo_cmd += f" --seed {CONFIG['seed']}"

if CONFIG['enable_oom_recovery']:
    dpo_cmd += " --enable-oom-recovery"

print("DPO command:")
print(dpo_cmd)

In [ ]:
# Start DPO training
!{dpo_cmd}

---
## Stage 5: LoRA Fine-Tuning (Optional)

Domain-specific adaptation using LoRA for efficient fine-tuning.

In [ ]:
# LoRA fine-tuning (optional)
# Uncomment to run LoRA training

# !python scripts/10_lora_finetune.py

---
## Evaluation

Evaluate the trained model on various benchmarks.

In [ ]:
# Run full evaluation suite
CHECKPOINT_TO_EVAL = "checkpoints/dpo_final"  # Change as needed

!python scripts/11_evaluate.py {CHECKPOINT_TO_EVAL}

In [ ]:
# Check promotion gates
# Verify model meets quality thresholds

STAGE_TO_CHECK = "dpo"  # Options: pretrain, sft, dpo

!python scripts/12_check_gates.py {STAGE_TO_CHECK}

---
## Monitoring & Utilities

In [ ]:
# Monitor GPU utilization
!nvidia-smi

In [ ]:
# List all checkpoints
!bash scripts/checkpoint_manager.sh list

In [ ]:
# Show disk usage
!bash scripts/checkpoint_manager.sh disk-usage

In [ ]:
# Cleanup old checkpoints (keep latest 3)
# Uncomment to run

# !bash scripts/checkpoint_manager.sh cleanup pretrain 3

---
## Model Inference

Test the trained model with interactive generation.

In [ ]:
# Load the trained model for inference
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch

MODEL_PATH = "checkpoints/dpo_final"  # Change to your checkpoint

print(f"Loading model from {MODEL_PATH}...")
tokenizer = AutoTokenizer.from_pretrained("configs/tokenizer")
model = AutoModelForCausalLM.from_pretrained(
    MODEL_PATH,
    torch_dtype=torch.bfloat16,
    device_map="auto"
)
print("Model loaded!")

In [ ]:
# Generate text
def generate(prompt, max_new_tokens=256, temperature=0.7):
    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
    
    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_new_tokens=max_new_tokens,
            temperature=temperature,
            do_sample=True,
            top_p=0.9,
            pad_token_id=tokenizer.eos_token_id,
        )
    
    response = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return response

# Test generation
prompt = "Explain machine learning in simple terms:"
print(f"Prompt: {prompt}\n")
print(f"Response: {generate(prompt)}")

In [ ]:
# Interactive generation cell
# Modify the prompt and run to test different inputs

PROMPT = "Write a Python function to calculate fibonacci numbers:"

print(f"Prompt: {PROMPT}\n")
print("=" * 50)
print(generate(PROMPT, max_new_tokens=512))

---
## Training Summary

After completing all stages, review the training summary.

In [ ]:
# Generate training report
import os
import json

print("=" * 60)
print("TRAINING PIPELINE SUMMARY")
print("=" * 60)

stages = [
    ('Pretrain', 'checkpoints/pretrain_final'),
    ('SFT', 'checkpoints/sft_final'),
    ('DPO', 'checkpoints/dpo_final'),
    ('LoRA', 'checkpoints/lora_final'),
]

print("\nCheckpoint Status:")
for name, path in stages:
    if os.path.exists(path):
        # Get checkpoint size
        size = sum(os.path.getsize(os.path.join(path, f)) for f in os.listdir(path) if os.path.isfile(os.path.join(path, f)))
        size_gb = size / (1024**3)
        print(f"  {name}: COMPLETE ({size_gb:.2f} GB)")
    else:
        print(f"  {name}: Not completed")

print("\nEvaluation Results:")
eval_path = "evals/"
if os.path.exists(eval_path):
    for f in os.listdir(eval_path):
        if f.endswith('.json'):
            with open(os.path.join(eval_path, f)) as file:
                results = json.load(file)
                print(f"  {f}: {results}")
else:
    print("  No evaluation results found. Run evaluation first.")

print("\n" + "=" * 60)